In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/


Mounted at /content/drive
/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL


In [ ]:
!git clone https://github.com/KarhouTam/SCAFFOLD-PyTorch.git
%cd SCAFFOLD-PyTorch


Cloning into 'SCAFFOLD-PyTorch'...
remote: Enumerating objects: 151, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 151 (delta 47), reused 29 (delta 29), pack-reused 82 (from 1)
Receiving objects: 100% (151/151), 40.72 KiB | 1.63 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/SCAFFOLD-PyTorch


In [16]:
import sys
print(sys.version)


3.12.3 (main, Jun 18 2025, 17:59:45) [GCC 13.3.0]


In [17]:
!pip install -r requirements.txt

In [3]:
%cd SCAFFOLD-PyTorch
!pwd

[Errno 2] No such file or directory: 'SCAFFOLD-PyTorch'
/home/kth0910/jupyter-env/SCAFFOLD-PyTorch/SCAFFOLD-PyTorch
/home/kth0910/jupyter-env/SCAFFOLD-PyTorch/SCAFFOLD-PyTorch


In [18]:
!nvidia-smi
import torch
print(torch.cuda.is_available())        # True이면 사용 가능
print(torch.cuda.get_device_name(0))    # GPU 이름


Tue Jun 24 02:35:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.02              Driver Version: 560.94         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2060        On  |   00000000:07:00.0  On |                  N/A |
| 49%   40C    P8             12W /  160W |     544MiB /   6144MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
!python ./data/utils/run.py \
    --dataset emnist \
    --type user \
    --emnist_split balanced \
    --client_num_in_total 100 \
    --alpha 10 \
    --client_num_in_each_pickles 10 \
    --root ./datasets



=== Generating EMNIST dataset with similarity=0 ===
100%|████████████████████████████████████████| 562M/562M [00:08<00:00, 70.0MB/s]

=== Generating EMNIST dataset with similarity=10 ===
100%|████████████████████████████████████████| 562M/562M [00:09<00:00, 56.4MB/s]

=== Generating EMNIST dataset with similarity=100 ===
100%|████████████████████████████████████████| 562M/562M [00:12<00:00, 46.3MB/s]


In [ ]:
import os
import pandas as pd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# 실험 설정 CSV 파일 경로
csv_path = "experiment_config_from_paper.csv"
df = pd.read_csv(csv_path)

# 결과 저장용 리스트
results = []

# 알고리즘별 실행 경로
algo_to_script = {
    "sgd": "src/server/fedavg.py",  # 1 client로 설정되어 있으면 SGD로 동작
    "fedavg": "src/server/fedavg.py",
    "fedprox": "src/server/fedprox.py",
    "scaffold": "src/server/scaffold.py",
}

# 출력 로그 저장 디렉토리
log_dir = "logs"
os.makedirs(log_dir, exist_ok=True)

# 실험 실행 루프
for idx, row in df.iterrows():
    algorithm = row['algorithm']

    similarity = int(row['similarity'])
    epochs = int(row['epochs'])
    batch_fraction = float(row['batch_fraction'])
    sim_path = f"data/datasets/emnist_sim{similarity}/emnist"
    print(f"[실행 중] {algorithm} / similarity={similarity}% / epochs={epochs} / batch_fraction={batch_fraction}")

    # 로그 파일 경로 구성
    filename = f"{algorithm}_sim{similarity}_ep{epochs}.log"
    log_path = os.path.join(log_dir, filename)

    # 실행
    cmd = [
        "python", "-m", f"server.{algorithm if row['algorithm']!='sgd' else 'fedavg'}",
        "--config_idx", str(idx),
        "--data_root", sim_path
    ]
    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        env={**os.environ, "PYTHONPATH": "src:data"}
        )

    # 로그 저장
    with open(log_path, "w", encoding="utf-8") as f:
        f.write(result.stdout)
        f.write("\n" + "=" * 80 + "\n")
        f.write(result.stderr)

    # 수렴 라운드 찾기
    round_str = next((line for line in result.stdout.splitlines() if "Reached 0.5 acc" in line), None)
    if round_str:
        rounds = int([x for x in round_str.split() if x.isdigit()][-1])
    else:
        rounds = -1 # 수렴 실패 시 -1로 기록

    results.append({
        "algorithm": algorithm,
        "similarity": similarity,
        "epochs": epochs,
        "rounds": rounds
    })

# 결과 저장
results_df = pd.DataFrame(results)
results_df.to_csv("experiment_results.csv", index=False)


[실행 중] sgd / similarity=0% / epochs=1 / batch_fraction=1.0


In [ ]:
import sys
import os

# 현재 작업 디렉토리를 기준으로 절대 경로 생성
project_root = os.path.abspath("src")
data_dir = os.path.abspath("data")

# sys.path에 추가
sys.path.append(project_root)
sys.path.append(data_dir)

print("[sys.path] 추가 완료")
print(sys.path)

[sys.path] 추가 완료
['/content', '/env/python', '/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/usr/local/lib/python3.11/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.11/dist-packages/IPython/extensions', '/usr/local/lib/python3.11/dist-packages/setuptools/_vendor', '/root/.ipython', '/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/SCAFFOLD-PyTorch/src', '/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/SCAFFOLD-PyTorch/data', '/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/SCAFFOLD-PyTorch/src', '/content/drive/MyDrive/2025KMU-FL/2025-KMU-industry-academic-cooperation-FL/SCAFFOLD-PyTorch/data']
